# Visualize Notebook
In this notebook the Gold Dataframe will be read and extract information of it. The objective is to see the correlations between the variables and the GDP and also what countries have the highest correlation value.


## Imports
Start importing all the libraries and also the methods of pvalue and search indicators that will be used later in the notebook.

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import shapiro, pearsonr
import os
import statistics
import seaborn as sns
from scipy.stats import norm

import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import AppLayout, Button, GridspecLayout
from ipywidgets import interact, interact_manual
from Project.Utils.visualize import  search


## Correlation dataframe.
This dataframe is the main piece of the notebook. Consists in generating for every country the correlation matrix for it and saving only the correlation value of the different variables with the GDP. Later on is concatenated and generates the following result:

In [33]:
#One dataframe per country

df= pd.read_csv (os.getcwd()+'/Output/GoldDataframe.csv')
corr_df = pd.DataFrame()
corr_df.index.names = ['Country']
aux_df = pd.DataFrame()

#List all the countries, none repeated
countries = set(df['Country'].to_list())

country_dict = {}
corr_dict = {}

for country in countries:

    #Get the DataFrame for a given country
    country_df = df.loc[df['Country'] == country]

    #Correlation matrix for that country
    country_corr_df = country_df.corr()

    #Significance for the correlations
    pval = country_corr_df.corr(method = lambda x, y: pearsonr(x, y)[1]) - np.eye(*country_corr_df.shape)
    p = pval.applymap(lambda x: 1 if x < 0.05 else np.NaN)
    country_corr_df = country_corr_df * p

    #Trim it into a single row
    country_corr_df = country_corr_df.rename(columns = {'GDP': country}).drop(index = ['Year', 'GDP'])

    #Add the row to a new DataFrame with the correlations for each country
    corr_df = pd.concat([corr_df, country_corr_df[country]], axis = 1)

#Transpose the resulting DataFrame to have the desired format and show it
corr_df = corr_df.transpose()
corr_df.to_csv(os.getcwd()+'/Output/Corr_DF_Test.csv')

corr_df.to_csv(os.getcwd()+'/Output/Corr_Test.csv')

#country_corr_df.corr(method = lambda x, y: pearsonr(x, y)[1]) - np.eye(*country_corr_df.shape).applymap(lambda x: 1 if x < 0.05 else np.NaN)

pval.to_csv(os.getcwd()+'/Output/PVal_Test.csv')
p.to_csv(os.getcwd()+'/Output/P_Test.csv')

df.loc[df['Country'] == 'Spain'].corr().to_csv(os.getcwd()+'/Output/Country_Corr_Test.csv')
corr_df.loc[['Spain']]


AgriShareGDP  CreditToAgriFishForest  EmploymentRural  TotalAgri  \
Malaysia        -0.057994                0.055144              NaN   0.860665   
Spain           -0.924602               -0.082207        -0.083709   0.891549   
New Zealand     -0.390130                0.142875              NaN   0.906371   
Czechia         -0.622233                0.492807         0.722958   0.783382   
China                 NaN                     NaN              NaN   0.949529   
...                   ...                     ...              ...        ...   
Ethiopia        -0.742794               -0.045399              NaN   0.856816   
Colombia        -0.928724                     NaN         0.758967   0.641759   
Poland          -0.510488                     NaN              NaN   0.927059   
Bahrain         -0.780047               -0.702905              NaN        NaN   
Lithuania       -0.835335                     NaN        -0.788530   0.912241   

             % Soldiers  Birth Rate  Death Rate  Homicides  Life Expectancy  \
Malaysia      -0.944179   -0.854599    0.852503        NaN         0.961953   
Spain         -0.828939   -0.071199   -0.466823  -0.743388         0.694753   
New Zealand   -0.934430   -0.724168   -0.375034  -0.810743         0.982542   
Czechia       -0.701305         NaN         NaN        NaN              NaN   
China         -0.860912   -0.362204    0.776483  -0.927046         0.982323   
...                 ...         ...         ...        ...              ...   
Ethiopia      -0.660334   -0.910035   -0.878882        NaN         0.914411   
Colombia       0.634779   -0.901693    0.659420  -0.837504         0.859863   
Poland        -0.424615    0.491552    0.839589  -0.947862         0.877672   
Bahrain       -0.952605   -0.978124   -0.823707        NaN         0.974140   
Lithuania      0.706347    0.574701    0.867250  -0.837935         0.746119   

             Maternal Death Risk  ...  % Population Growth  \
Malaysia               -0.819080  ...            -0.964197   
Spain                  -0.236050  ...            -0.146714   
New Zealand            -0.905752  ...             0.399911   
Czechia                      NaN  ...                  NaN   
China                  -0.868190  ...            -0.550435   
...                          ...  ...                  ...   
Ethiopia               -0.889476  ...            -0.845126   
Colombia               -0.874104  ...            -0.678381   
Poland                 -0.930270  ...             0.331447   
Bahrain                -0.912868  ...            -0.495596   
Lithuania              -0.812293  ...             0.071064   

             % Rural Population  Civil Liberties  Freedom of Expression  \
Malaysia              -0.970563         0.619590               0.613580   
Spain                 -0.634473        -0.030631              -0.230272   
New Zealand           -0.658080        -0.445694              -0.761484   
Czechia                     NaN         0.085240              -0.565038   
China                 -0.983006        -0.713696              -0.956523   
...                         ...              ...                    ...   
Ethiopia              -0.989688         0.534652               0.159649   
Colombia              -0.856736         0.782616               0.653603   
Poland                 0.905010        -0.507959              -0.510648   
Bahrain               -0.886818        -0.874272              -0.715059   
Lithuania             -0.633390         0.197869               0.077926   

             % Healthcare Investment  % Employment Industry  \
Malaysia                    0.888805              -0.878161   
Spain                       0.794086              -0.592126   
New Zealand                 0.892840              -0.751467   
Czechia                     0.715060              -0.599416   
China                       0.825297               0.767233   
...                              ...                    ...   
Ethi

In [32]:
country_corr_df = df.loc[df['Country'] == 'Spain'].corr()
pval = country_corr_df.corr(method = lambda x, y: pearsonr(x, y)[1]) - np.eye(*country_corr_df.shape)
p = pval.applymap(lambda x: 1 if x < 0.05 else np.NaN)
res_corr_df = country_corr_df * p

country_corr_df.to_csv(os.getcwd()+'/Output/Corr_Test.csv')
pval.to_csv(os.getcwd()+'/Output/PVal_Test.csv')
res_corr_df.to_csv(os.getcwd()+'/Output/Res_Test.csv')

In [17]:
#Get the DataFrame for a given country
country_df = df.loc[df['Country'] == 'Spain']

#Correlation matrix for that country
aux_corr_df = country_df.corr().rename(columns = {'GDP': 'Spain'}).drop(index = ['Year', 'GDP'])
pval = country_df.corr(method = lambda x, y: pearsonr(x, y)[1])
p = pval.applymap(lambda x: 1 if x < 0.05 else np.NaN)

corr_df = corr_df['Spain']
p = p['GDP']

aux_corr_df = aux_corr_df * p

#corr_df = corr_df.rename(columns = {'GDP': 'Spain'}).drop(index = ['Year', 'GDP'])
aux_corr_df
#print(country_corr_df['Spain'])
#print(country_df.corr()['GDP'])
#print(p)


% Education Expenditure         NaN
% Employment Industry     -0.592126
% Healthcare Investment    0.794086
% Men Employment                NaN
% Population Growth             NaN
% Rural Population        -0.634473
% Soldiers                -0.828939
% Women Employment         0.927052
AgriShareGDP              -0.924602
Birth Rate                      NaN
Civil Liberties                 NaN
CreditToAgriFishForest          NaN
Death Rate                -0.466823
EmploymentRural                 NaN
Freedom of Expression           NaN
GDP                             NaN
Homicides                 -0.743388
Infant Mortality          -0.734420
Life Expectancy            0.694753
Maternal Death Risk             NaN
Population                 0.899727
TotalAgri                  0.891549
Year                            NaN
dtype: float64

In [30]:
ind = 'Population'
t = 0.7
pos_corr = corr_df.drop(corr_df.columns.difference([ind]), axis = 1).loc[corr_df[ind] >= t].sort_values(by = ind, axis = 0, ascending = False)
neg_corr = corr_df.drop(corr_df.columns.difference([ind]), axis = 1).loc[corr_df[ind] <= -t].sort_values(by = ind, axis = 0, ascending = True)
neg_corr

,Population
Hungary,-0.825989
Romania,-0.938846
Armenia,-0.787782
Serbia,-0.912291
Georgia,-0.955508
Latvia,-0.863641
Poland,-0.827611
Bulgaria,-0.951117
Estonia,-0.919445
Palau,-0.721353


In [22]:
import ipywidgets as widgets
import plotly.express as px

from IPython.display import display_html
import plotly.graph_objects as go
from plotly.subplots import make_subplots


indicator = widgets.SelectMultiple(
    options = corr_df.columns.tolist(),
    value = ['AgriShareGDP'],
    description='Indicator',
    disabled=False,
    layout = Layout(width='50%', height='80px')
)

#country_table = widgets.

def globalGrapgh(indicator):
    ind = indicator[0]
    N = 10
    fig = px.choropleth(corr_df, locations = corr_df.index, locationmode='country names', 
                        color= indicator[0],projection="natural earth",
                        color_continuous_scale='Reds',
                        width=700, height=500)

    pos_corr = corr_df.drop(corr_df.columns.difference([ind]), axis = 1).sort_values(by = ind, axis = 0, ascending = False).head(n = N)
    neg_corr = corr_df.drop(corr_df.columns.difference([ind]), axis = 1).sort_values(by = ind, axis = 0, ascending = True).head(n = N)

    #print(fig.get_subplot(1, 1))

    #subplots = make_subplots(rows = 1, cols = 3, specs = [[{'type': 'choropleth'}, {'type': 'table'}, {'type': 'table'}]], subplot_titles = ['Map', 'Direct Correlation', 'Inverse Correlation'])
    #subplots.add_trace(fig, row = 0, col = 0)

    fig.update(layout_coloraxis_showscale=True)
    fig.show()
    
    #widgets.HBox([pos_corr.style, neg_corr.style])
    pos_styler = pos_corr.style.set_table_attributes("style='display:inline'").set_caption('Direct correlation')
    neg_styler = neg_corr.style.set_table_attributes("style='display:inline'").set_caption('Inverse correlation')

    space = "\xa0" * 10
    display_html(pos_styler._repr_html_()+ space  + neg_styler._repr_html_(), raw=True)

    #print(pos_corr)
    #print(neg_corr)

    #grid[0, 0] = fig
    #grid[0, 1] = pos_corr
    #grid[0, 2] = neg_corr    
    
widgets.interactive(globalGrapgh, indicator = indicator)

interactive(children=(SelectMultiple(description='Indicator', index=(0,), layout=Layout(height='80px', width='…

## Country Indicators
In this codeblock a widged is implmented to generate a table with the indicators. In order to make it work just select the country from the dropdown and procede to establasih the threshold. The default value for the threshold is 0.7, which we consider to be the minimum to consider an indicator correlated to the GDP. To analyze the results:
H0: the indicator and the GDP are uncorrelated.
H1: the indicator and the GDP are correlated.
The p-value represents the probability that your data would have arisen if the null hypothesis were true. If it's lower than 0.05 then it has no statistical signifcance.

If the both correlations are greater than 0.7 or -0.7 and the p-values are < 0.05 the result 

In [4]:
def tableOut(Threshold, Country):

    df = search(Threshold, 'Country', Country)
    if df.empty:
        return print("No indicators have been found.")

    left = pd.Series([0.05, 0.05], index=['P-value Pearson', 'P-value Spearman'])
    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df =df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
        .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman']) 
    
    display(df)

    

@interact(
    Country = sorted(corr_df.index.tolist()),
    Threshold = (0, 1, 0.05))
def g(Country = 'Afghanistan', Threshold = 0.7):
    return tableOut(Threshold,Country)

    
        

interactive(children=(Dropdown(description='Country', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra',…

## Region Indicators


In [5]:
def tableOut2(Threshold, Region):

    df = search(Threshold, 'Region', Region)
    if df.empty:
        return print("No indicators have been found.")

    left = pd.Series([0.05, 0.05], index=['P-value Pearson', 'P-value Spearman'])
    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df =df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
        .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman']) 
    
    display(df)



@interact(
    Region = set(df['Region'].to_list()),
    Threshold = (0, 1, 0.05))
def g(Region = 'East Asia and Pacific', Threshold = 0.7):
    return tableOut2(Threshold, Region)

    

interactive(children=(Dropdown(description='Region', index=2, options=('Europe and Central Asia', 'South Asia'…

## Global Indicators


In [6]:
def tableOut3(Threshold):

    df = search(Threshold, 'Global')
    if df.empty:
        return print("No indicators have been found.")

    left = pd.Series([0.05, 0.05], index=['P-value Pearson', 'P-value Spearman'])
    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df =df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
        .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman']) 
    
    display(df)

    

@interact(
    Threshold = (0, 1, 0.05))
def g(Threshold = 0.7):
    return tableOut3(Threshold)


interactive(children=(FloatSlider(value=0.7, description='Threshold', max=1.0, step=0.05), Output()), _dom_cla…